In [1]:
import os
os.environ["TQDM_DISABLE"] = "1"

import tqdm
from tqdm import tqdm as tqdm_class

_original_init = tqdm_class.__init__
def _patched_init(self, *args, **kwargs):
    kwargs["disable"] = True
    _original_init(self, *args, **kwargs)
tqdm_class.__init__ = _patched_init
tqdm.tqdm = tqdm_class

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
import sys
import time
import logging
logging.getLogger("anomalib").setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

from tqdm import tqdm
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image

In [2]:
try:
    import google.colab
    from google.colab import drive
    !uv pip install anomalib
    !uv pip install open-clip-torch
    drive.mount('/content/drive', force_remount=True)
    PROJECT_ROOT = Path('/content/drive/Othercomputers/Mac/multiModal_anomaly_report') # 본인 경로 수정: Mac/Window
except ImportError:
    PROJECT_ROOT = Path.cwd().parents[1]

os.chdir(PROJECT_ROOT) # 현재 경로 수정
print(f"Current working directory: {os.getcwd()}")

Using Python 3.12.12 environment at: /usr
Audited 1 package in 110ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 96ms
Mounted at /content/drive
Current working directory: /content/drive/Othercomputers/Mac/multiModal_anomaly_report


### Check config

In [3]:
# TODO: PATH 설정

# dataset path
DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"

# config.yaml path
CONFIG_ROOT = PROJECT_ROOT / "configs"
RUNTIME_CONFIG_ROOT = CONFIG_ROOT / "runtime.yaml"
EVAL_CONFIG_ROOT = CONFIG_ROOT / "eval.yaml"

# output path
OUTPUT_ROOT = PROJECT_ROOT / "output"

# check (선택사항)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Root: {DATA_ROOT}")
print(f"Config Root: {RUNTIME_CONFIG_ROOT}")

Project Root: /content/drive/Othercomputers/Mac/multiModal_anomaly_report
Data Root: /content/drive/Othercomputers/Mac/multiModal_anomaly_report/dataset/MMAD
Config Root: /content/drive/Othercomputers/Mac/multiModal_anomaly_report/configs/runtime.yaml


In [4]:
from src.utils import load_config, load_json, load_csv

# tqdm bar off
os.environ["TQDM_DISABLE"] = "1"

# TODO: runtime_config.yaml 확장/수정
config = load_config(RUNTIME_CONFIG_ROOT)
config

{'data': {'root': 'dataset/MMAD',
  'output_root': './output',
  'image_size': [256, 256],
  'datasets': ['MVTec-LOCO']},
 'anomaly': {'model': 'efficientad',
  'patchcore': {'backbone': None,
   'layers': None,
   'coreset_sampling_ratio': None},
  'winclip': {'class_name': None, 'k_shot': None, 'scales': None},
  'efficientad': {'model_size': None,
   'teacher_out_channels': None,
   'visualizer': False,
   'evaluator': True,
   'imagenet_dir': 'imagenette/train'}},
 'training': {'resume': True,
  'train_batch_size': None,
  'eval_batch_size': None,
  'num_workers': 6,
  'max_epochs': 100},
 'engine': {'accelerator': 'auto',
  'enable_progress_bar': False,
  'logger': 'wandb'},
 'wandb': {'project': 'Anomaly_Detection', 'run_name': 'resume-90'},
 'output': {'save_json': True}}

### Train

In [ ]:
from scripts.train_anomalib import Anomalibs

runner = Anomalibs() # default path: RUNTIME_CONFIG_ROOT
runner.fit_all()

TrainAnomalib - INFO - Initialized - model: efficientad, device: cuda
TrainAnomalib - INFO - Starting fit_all: 5 categories
TrainAnomalib - INFO - [1/5] Training: MVTec-LOCO/breakfast_box...
TrainAnomalib - INFO - Fitting efficientad - MVTec-LOCO/breakfast_box
TrainAnomalib - INFO - Resume is true. Found checkpoint, resuming from: output/EfficientAd/MVTec-LOCO/breakfast_box/v0/model.ckpt


Device: NVIDIA A100-SXM4-40GB

[1/5] Training: MVTec-LOCO/breakfast_box...


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moonstalker9010 (moonstalker9010-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO:pytorch_lightning.utilities.rank_zero:You 

wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/Othercomputers/Mac/multiModal_anomaly_report/output/EfficientAd/MVTec-LOCO/breakfast_box/v0/model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor  │ PreProcessor     │      0 │ train │     0 │
│ 1 │ post_processor │ PostProcessor    │      0 │ train │     0 │
│ 2 │ evaluator      │ Evaluator        │      0 │ train │     0 │
│ 3 │ model          │ EfficientAdModel │  8.1 M │ train │     0 │
└───┴────────────────┴──────────────────┴────────┴───────┴───────┘

...

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/Othercomputers/Mac/multiModal_anomaly_report/output/EfficientAd/MVTec-LOCO/splicing_connectors/v0/model.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor  │ PreProcessor     │      0 │ train │     0 │
│ 1 │ post_processor │ PostProcessor    │      0 │ train │     0 │
│ 2 │ evaluator      │ Evaluator        │      0 │ train │     0 │
│ 3 │ model          │ EfficientAdModel │  8.1 M │ train │     0 │
└───┴────────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 8.1 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 8.1 M                                                                                                
Total estimated model params size (MB): 32                                                                         
Modules in train mode: 48                                                                                          
Modules in eval mode: 7                                                                                            
Total FLOPs: 0

INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/drive/Othercomputers/Mac/multiModal_anomaly_report/output/EfficientAd/MVTec-LOCO/splicing_connectors/v0/model.ckpt


[Epoch 11/100] | loss=0.8980
[Epoch 12/100] | loss=0.8334
[Epoch 13/100] | loss=0.8442
[Epoch 14/100] | loss=0.7752
[Epoch 15/100] | loss=0.7530
[Epoch 16/100] | loss=0.7161
[Epoch 17/100] | loss=0.6894
[Epoch 18/100] | loss=0.7105
[Epoch 19/100] | loss=0.6494
[Epoch 20/100] | loss=0.6648


### Predict

In [ ]:
from scripts.train_anomalib import Anomalibs

runner = Anomalibs() # default path: RUNTIME_CONFIG_ROOT
predictions = runner.predict_all()
predictions

In [ ]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    jaccard_score
)

results = []
for category, batches in predictions.items():
    # Image-level
    y_true = np.concatenate([b["gt_label"].cpu().numpy() for b in batches])
    y_score = np.concatenate([b["pred_score"].cpu().numpy() for b in batches])
    y_pred = (y_score >= 0.5).astype(int)

    metrics = {
        "Category": category,
        "AUROC": round(roc_auc_score(y_true, y_score), 4),
        "Accuracy": round(accuracy_score(y_true, y_pred), 4),
        "Precision": round(precision_score(y_true, y_pred, zero_division=0), 4),
        "Recall": round(recall_score(y_true, y_pred, zero_division=0), 4),
        "F1": round(f1_score(y_true, y_pred, zero_division=0), 4),
        "N_samples": len(y_true)
    }

    # Pixel-level (mask가 있는 경우만)
    if "gt_mask" in batches[0] and batches[0]["gt_mask"] is not None:
        gt_masks = torch.cat([b["gt_mask"] for b in batches]).int()
        pred_masks = torch.cat([(b["anomaly_map"] > 0.5).int() for b in batches])

        metrics["Dice"] = round(
            f1_score(
                gt_masks.flatten().cpu().numpy(),
                pred_masks.flatten().cpu().numpy(),
                zero_division=0
            ), 4
        )
        metrics["IoU"] = round(
            jaccard_score(
                gt_masks.flatten().cpu().numpy(),
                pred_masks.flatten().cpu().numpy(),
                average="binary",
                zero_division=0
            ), 4
        )

    results.append(metrics)

metrics_df = pd.DataFrame(results).set_index("Category")
avg_row = metrics_df.drop(columns=["N_samples"]).mean().round(4)
avg_row["N_samples"] = metrics_df["N_samples"].sum()
metrics_df.loc["Average"] = avg_row

metrics_df

In [ ]:
from src.visual.plot import heatmap_plot

metrics_trans = metrics_df.drop('Average').drop(columns='N_samples')
heatmap_plot(
    metrics_trans,
    figsize=(10, 10),
    cmap='RdYlGn',
    annot=True,
    fmt='.4f',
    linewidths=0.5,
    title='EfficientAd Performance by Category',
    rotation_x=45,
    rotation_y=0
)

In [ ]:
from src.visual.plot import visualize_prediction, visualize_predictions_grid, visualize_all_predictions

batch = predictions["GoodsAD/cigarette_box"][0]
visualize_prediction(batch, idx=0)

# 여러 샘플 보려면 반복
for i in range(4):
    visualize_prediction(batch, idx=i)

# for category, batches in predictions.items():
#     print(f"\n=== {category} ===")
#     count = 0
#     for batch in batches:
#         for i in range(len(batch["image"])):
#             visualize_prediction(batch, idx=i, title=f"{category}")
#             count += 1
#             if count >= 4:  # 카테고리당 4개만
#                 break
#         if count >= 4:
#             break

In [ ]:
from src.visual.plot import anomaly_grid_from_dir

for category_key in predictions.keys():  # "GoodsAD/drink_bottle" 형식
    dataset, category = category_key.split("/")
    OUTPUT_IMG_ROOT = OUTPUT_ROOT / "Efficientad" / dataset / category / "v0"

    if OUTPUT_IMG_ROOT.exists():
        print(f"{dataset}/{category}")
        anomaly_grid_from_dir(OUTPUT_IMG_ROOT, n_samples=1, n_cols=1)
    else:
        print(f"{dataset}/{category} - 이미지 없음: {OUTPUT_IMG_ROOT}")

### Resume training

In [ ]:
config

In [ ]:
from scripts.train_anomalib import Anomalibs

runner = Anomalibs() # default path: RUNTIME_CONFIG_ROOT
runner.fit_all()